# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

# Access maps with unique API key
gmaps.configure(api_key=g_key)


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:
city_df = pd.read_csv('cities.csv', index_col='City_ID')
city_df.head()

,country_code,city,lat,lng,date,max_temp,humidity,wind_speed,cloudiness
City_ID,,,,,,,,,
0,mg,taolanaro,-55.068682,61.721037,1651142411,1.32,73,73,71
1,pf,mataura,-70.082829,-145.302727,1651142411,-2.23,97,97,100
2,sj,barentsburg,88.974684,-6.388620,1651142411,-22.72,98,98,41
3,cn,zhumadian,32.903080,113.484903,1651142411,8.61,89,89,100
4,au,carnarvon,-29.826881,91.805235,1651142412,19.14,66,66,3


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
#Create a map using state centroid coordiantes to set markers 
locations = city_df[['lat','lng']]
w = city_df['humidity'].astype(float)

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=w,
                            dissipating=False, max_intensity=100,
                            point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
#set weather criteria parameter
max_cloud = 20
max_humidity = 20
max_windspeed = 25
max_temp = 25

#store into new dataframe
hotel_df = city_df.loc[(city_df['cloudiness'] <= max_cloud) & (city_df['humidity'] <= max_humidity) & (city_df['wind_speed'] <= max_windspeed)& (city_df['max_temp'] <= max_temp)].reset_index()

hotel_df.reset_index(inplace=True)

hotel_df

,index,City_ID,country_code,city,lat,lng,date,max_temp,humidity,wind_speed,cloudiness
0,0,120,cn,kashi,40.136252,77.381166,1651142438,21.02,9,9,0
1,1,495,cn,taonan,45.329322,123.227431,1651142514,12.07,17,17,3
2,2,509,ar,tucuman,-26.506221,-66.208145,1651142517,2.76,14,14,0
3,3,618,cn,altay,46.498762,88.895528,1651142536,15.73,14,14,4
4,4,626,cl,vallenar,-28.114070,-70.463953,1651142538,13.53,20,20,0
5,5,750,cn,zhengjiatun,43.489723,123.645296,1651142561,13.29,18,18,3
6,6,1472,bo,challapata,-19.612848,-67.830126,1651142694,5.05,14,14,9
7,7,1486,mx,chihuahua,29.160635,-105.814747,1651142696,18.09,20,20,0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    #"keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["lat"]
    lng = row["lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    #print(json.dumps(name_address["results"][0]["name"], indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]

    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


,index,City_ID,country_code,city,lat,lng,date,max_temp,humidity,wind_speed,cloudiness,Hotel Name
0,0,120,cn,kashi,40.136252,77.381166,1651142438,21.02,9,9,0,NaN
1,1,495,cn,taonan,45.329322,123.227431,1651142514,12.07,17,17,3,NaN
2,2,509,ar,tucuman,-26.506221,-66.208145,1651142517,2.76,14,14,0,NaN
3,3,618,cn,altay,46.498762,88.895528,1651142536,15.73,14,14,4,NaN
4,4,626,cl,vallenar,-28.114070,-70.463953,1651142538,13.53,20,20,0,NaN
5,5,750,cn,zhengjiatun,43.489723,123.645296,1651142561,13.29,18,18,3,Grand Hyatt Hotel
6,6,1472,bo,challapata,-19.612848,-67.830126,1651142694,5.05,14,14,9,NaN
7,7,1486,mx,chihuahua,29.160635,-105.814747,1651142696,18.09,20,20,0,NaN


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country_code}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations_1 = hotel_df[["lat", "lng"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations_1,info_box_content=hotel_info)
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))